---

# <center> *__`ShadowTap`__*

---

In [21]:
import platform
import subprocess
import ipaddress
from scapy.all import ARP, Ether, srp
import rich
from rich.console import Console
from rich.table import Table

console = Console()

In [17]:
def gen(text: str, style: str):
    """This program is used to generate strings to print in sytl
    Eg - print_(gen("Error occured :( , failure not found!", 'bold #ff471a'))"""
    output = "[{}]{}[/{}]".format(style, text, style)
    return output

console.print(gen("ShadowTap Builder Initiated...", "bold #33cc33"))
marker = gen(">>>", "bold #ff0cde")

ShadowTap Builder Initiated...

In [18]:
def detect_os():
    """
    Detect the operating system the script is running on.
    
    Returns:
        str: The lowercase name of the OS ('linux', 'windows', or 'unsupported').
    """
    os_name = platform.system().lower()
    console.print(gen(f"{marker} Detected OS: {gen(os_name, 'bold #33ff33')}", "bold #3399ff"))
    if os_name == 'linux' or os_name == 'windows':
        return os_name
    return 'unsupported'

detect_os()

>>> Detected OS: windows

'windows'

---

### *__`Concept Time : `__*

CIDR (Classless Inter-Domain Routing) is a method for allocating IP addresses and routing IP packets on networks. It's basically a way to represent an IP network range more efficiently than the old class-based system (like Class A/B/C networks).

#### Quick Breakdown:
- **Format**: It's written as an IP address followed by a slash and a number, e.g., `192.168.1.0/24`. 
  - The IP (like `192.168.1.0`) is the network prefix.
  - The number after the slash (e.g., `/24`) is the subnet mask in bits—telling how many bits are fixed for the network part vs. the host part.
- **Why it exists**: Back in the day, IP addresses were divided into rigid classes (e.g., Class C was always /24 with 256 addresses). CIDR lets you use variable-length subnet masks (VLSM) for more flexible sizing, reducing waste and helping with IP exhaustion.
- **How it works**: The /number indicates the number of bits in the subnet mask that are set to 1 (network bits). For example:
  - `/24` means the first 24 bits are for the network, leaving 8 bits for hosts (2^8 = 256 addresses, but minus network/broadcast = 254 usable).
  - `/16` would give 65,536 addresses (2^16), and so on.

#### Example in Your Context:
In the Python code we've been tweaking (like for ARP scanning), `192.168.1.0/24` means scanning all IPs from 192.168.1.0 to 192.168.1.255—your typical home WiFi subnet. It auto-fills based on your OS to grab the right range without hardcoding.

If you're diving deeper into networking for your pentest tool, CIDR is key for defining scan scopes without over-scanning.

In [19]:
def fetch_network_info_linux():
    """
    Fetch the local network CIDR and default gateway IP on Linux systems.
    
    Uses system commands to retrieve the default interface, local IP, subnet mask,
    and gateway. Computes the CIDR notation for the network range.
    
    Returns:
        dict: A dictionary with 'cidr' (str) and 'gateway' (str).
    
    Raises:
        ValueError: If unable to retrieve network information.
    """
    console.print(gen(f"{marker} Fetching network info on Linux...", "bold #3399ff"))
    try:
        # Get default route info
        cmd = "ip route get 8.8.8.8"
        output = subprocess.check_output(cmd.split()).decode()
        parts = output.split()
        gateway = parts[parts.index('via') + 1]
        iface = parts[parts.index('dev') + 1]
        console.print(gen(f"{marker} Default Gateway: {gen(gateway, 'bold #33ff33')}, Interface: {gen(iface, 'bold #33ff33')}", "bold #3399ff"))
        
        # Get IP and mask for the interface
        cmd = ["ip", "addr", "show", iface]
        output = subprocess.check_output(cmd).decode()
        for line in output.splitlines():
            if 'inet ' in line and 'scope global' in line:
                inet = line.strip().split()[1]  # e.g., '192.168.1.100/24'
                network = ipaddress.ip_interface(inet).network
                cidr = str(network)
                console.print(gen(f"{marker} Network CIDR: {gen(cidr, 'bold #33ff33')}", "bold #3399ff"))
                return {'cidr': cidr, 'gateway': gateway}
        raise ValueError("Could not find IP/mask for interface")
    except Exception as e:
        console.print(gen(f"{marker} Error fetching network info on Linux: {gen(str(e), 'bold #ff471a')}", "bold #ff471a"))
        raise ValueError(f"Failed to fetch network info on Linux: {e}")

# _____________________________________________________________________________________________________________________

def fetch_network_info_windows():
    """
    Fetch the local network CIDR and default gateway IP on Windows systems.
    
    Parses the output of 'ipconfig' to find the active network adapter with
    a default gateway, then computes the CIDR notation.
    
    Returns:
        dict: A dictionary with 'cidr' (str) and 'gateway' (str).
    
    Raises:
        ValueError: If unable to retrieve network information.
    """
    console.print(gen(f"{marker} Fetching network info on Windows...", "bold #3399ff"))
    try:
        output = subprocess.check_output('ipconfig').decode('latin-1')
        lines = output.splitlines()
        ip = None
        mask = None
        gateway = None
        for line in lines:
            line = line.strip()
            if 'IPv4 Address' in line:
                ip = line.split(':')[-1].strip()
                console.print(gen(f"{marker} IPv4 Address: {gen(ip, 'bold #33ff33')}", "bold #3399ff"))
            elif 'Subnet Mask' in line:
                mask = line.split(':')[-1].strip()
                console.print(gen(f"{marker} Subnet Mask: {gen(mask, 'bold #33ff33')}", "bold #3399ff"))
            elif 'Default Gateway' in line:
                gateway = line.split(':')[-1].strip()
                console.print(gen(f"{marker} Default Gateway: {gen(gateway, 'bold #33ff33')}", "bold #3399ff"))
                if gateway and '.' in gateway and ip and mask:
                    interface = f"{ip}/{mask}"
                    network = ipaddress.ip_interface(interface).network
                    cidr = str(network)
                    console.print(gen(f"{marker} Network CIDR: {gen(cidr, 'bold #33ff33')}", "bold #3399ff"))
                    return {'cidr': cidr, 'gateway': gateway}
        raise ValueError("No active network adapter with gateway found")
    except Exception as e:
        console.print(gen(f"{marker} Error fetching network info on Windows: {gen(str(e), 'bold #ff471a')}", "bold #ff471a"))
        raise ValueError(f"Failed to fetch network info on Windows: {e}")

In [20]:
fetch_network_info_windows()

>>> Fetching network info on Windows...

>>> IPv4 Address: 10.103.140.196

>>> Subnet Mask: 255.255.224.0

>>> Default Gateway: 10.103.128.1

>>> Network CIDR: 10.103.128.0/19

{'cidr': '10.103.128.0/19', 'gateway': '10.103.128.1'}

In [22]:
def perform_arp_scan(ip_range):
    """
    Perform an ARP scan on the given IP range to discover connected devices.
    
    Crafts and sends ARP request packets, collects responses, and returns a list
    of devices with their IP and MAC addresses.
    
    Args:
        ip_range (str): The network range in CIDR notation (e.g., '192.168.1.0/24').
    
    Returns:
        list: A list of dictionaries, each with 'ip' and 'mac' keys.
    """
    console.print(gen(f"{marker} Starting ARP scan on {gen(ip_range, 'bold #33ff33')}...", "bold #3399ff"))
    arp = ARP(pdst=ip_range)
    ether = Ether(dst="ff:ff:ff:ff:ff:ff")
    packet = ether / arp
    try:
        result = srp(packet, timeout=3, verbose=0)[0]
        
        devices = []
        for sent, received in result:
            devices.append({'ip': received.psrc, 'mac': received.hwsrc})
        
        console.print(gen(f"{marker} Scan complete. Found {gen(str(len(devices)), 'bold #33ff33')} devices.", "bold #3399ff"))
        return devices
    except Exception as e:
        console.print(gen(f"{marker} Error during ARP scan: {gen(str(e), 'bold #ff471a')}", "bold #ff471a"))
        raise

In [ ]:
def print_devices(devices, gateway):
    """
    Print the list of discovered devices in a formatted table, highlighting the router.
    
    Args:
        devices (list): List of device dictionaries with 'ip' and 'mac'.
        gateway (str): The IP address of the router/default gateway.
    """
    if not devices:
        console.print(gen(f"{marker} No devices discovered on the network.", "bold #ff471a"))
        return
    
    console.print(gen(f"{marker} Displaying discovered devices:", "bold #3399ff"))
    
    table = Table(show_header=True, header_style="bold #33cc33", border_style="dim")
    table.add_column("IP", style="bold #ffffff", width=15)
    table.add_column("MAC", style="bold #ffffff", width=17)
    table.add_column("Type", style="bold #ffffff", width=10)
    
    for device in devices:
        device_type = "(Router)" if device['ip'] == gateway else ""
        row_style = "on #333333" if device_type else ""
        table.add_row(
            gen(device['ip'], 'bold #33ff33'),
            gen(device['mac'], 'bold #33ff33'),
            gen(device_type, 'bold #ff0cde'),
            style=row_style
        )
    
    console.print(table)



>>> Detected OS: windows

>>> Fetching network info on Windows...

>>> IPv4 Address: 10.103.140.196

>>> Subnet Mask: 255.255.224.0

>>> Default Gateway: 10.103.128.1

...

>>> Scanning network: 10.103.128.0/19

>>> Default Gateway (Router): 10.103.128.1

>>> Starting ARP scan on 10.103.128.0/19...

>>> Error during ARP scan: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You 
may use conf.L3socket or conf.L3socket6 to access layer 3

>>> Unexpected error: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may 
use conf.L3socket or conf.L3socket6 to access layer 3. Ensure script is run with administrator/root privileges.

In [ ]:
if __name__ == "__main__":
    os_type = detect_os()
    if os_type == 'unsupported':
        console.print(gen("Unsupported OS. This script supports Linux and Windows only.", "bold #ff471a"))
    else:
        try:
            if os_type == 'linux':
                info = fetch_network_info_linux()
            elif os_type == 'windows':
                info = fetch_network_info_windows()
            
            ip_range = info['cidr']
            gateway = info['gateway']
            
            console.print(gen(f"{marker} Scanning network: {gen(ip_range, 'bold #33ff33')}", "bold #3399ff"))
            console.print(gen(f"{marker} Default Gateway (Router): {gen(gateway, 'bold #33ff33')}", "bold #3399ff"))
            
            devices = perform_arp_scan(ip_range)
            print_devices(devices, gateway)
        except ValueError as e:
            console.print(gen(f"{marker} Error: {gen(str(e), 'bold #ff471a')}", "bold #ff471a"))
        except Exception as e:
            console.print(gen(f"{marker} Unexpected error: {gen(str(e), 'bold #ff471a')}. Ensure script is run with administrator/root privileges.", "bold #ff471a"))

The error is a classic Scapy issue on Windows—it's complaining because it can't handle layer 2 packet sending/sniffing (needed for ARP scans) without a packet capture driver installed. WinPcap is the old one mentioned, but it's outdated and not recommended anymore.

### Quick Fix:
1. **Install Npcap** (the modern replacement for WinPcap, from the Nmap project):
   - Download it from [https://npcap.com/](https://npcap.com/) or directly [https://nmap.org/npcap/](https://nmap.org/npcap/).
   - Run the installer. During setup:
     - Check the box for "Install Npcap in WinPcap API-compatible Mode" if you have other tools that expect WinPcap.
     - No need for extra options unless you're doing wireless stuff.
   - Restart your machine after installation (important for the driver to load).

2. **Run your script as Administrator**:
   - Right-click your terminal/PowerShell/CMD and select "Run as administrator."
   - Or, if using an IDE like VS Code, run it elevated.

3. **Verify Scapy Setup**:
   - After installing, open a Python shell (as admin) and test:
     ```python
     from scapy.all import conf
     print(conf.use_pcap)  # Should be True if Npcap is detected
     ```
   - If it's False, double-check the installation.

This should resolve the layer 2 access issue. If you're actually on Kali Linux (as you mentioned earlier), this error shouldn't happen—Kali uses libpcap natively. But if you've switched to Windows for testing, yeah, Npcap is mandatory for Scapy's full features there.

If you still hit snags or want a code workaround (e.g., falling back to layer 3 sockets via `conf.L3socket`, though it's limited for ARP and might not scan the full range properly)

---
---
---

---

---

```python
def fetch_network_info_linux():
    """
    Fetch the local network CIDR and default gateway IP on Linux systems.
    
    Uses system commands to retrieve the default interface, local IP, subnet mask,
    and gateway. Computes the CIDR notation for the network range.
    
    Returns:
        dict: A dictionary with 'cidr' (str) and 'gateway' (str).
    
    Raises:
        ValueError: If unable to retrieve network information.
    """
    try:
        # Get default route info
        cmd = "ip route get 8.8.8.8"
        output = subprocess.check_output(cmd.split()).decode()
        parts = output.split()
        gateway = parts[parts.index('via') + 1]
        iface = parts[parts.index('dev') + 1]
        
        # Get IP and mask for the interface
        cmd = ["ip", "addr", "show", iface]
        output = subprocess.check_output(cmd).decode()
        for line in output.splitlines():
            if 'inet ' in line and 'scope global' in line:
                inet = line.strip().split()[1]  # e.g., '192.168.1.100/24'
                network = ipaddress.ip_interface(inet).network
                return {'cidr': str(network), 'gateway': gateway}
        raise ValueError("Could not find IP/mask for interface")
    except Exception as e:
        raise ValueError(f"Failed to fetch network info on Linux: {e}")

# _____________________________________________________________________________________________________________________

def fetch_network_info_windows():
    """
    Fetch the local network CIDR and default gateway IP on Windows systems.
    
    Parses the output of 'ipconfig' to find the active network adapter with
    a default gateway, then computes the CIDR notation.
    
    Returns:
        dict: A dictionary with 'cidr' (str) and 'gateway' (str).
    
    Raises:
        ValueError: If unable to retrieve network information.
    """
    try:
        output = subprocess.check_output('ipconfig').decode('latin-1')
        lines = output.splitlines()
        ip = None
        mask = None
        gateway = None
        for line in lines:
            line = line.strip()
            if 'IPv4 Address' in line:
                ip = line.split(':')[-1].strip()
            elif 'Subnet Mask' in line:
                mask = line.split(':')[-1].strip()
            elif 'Default Gateway' in line:
                gateway = line.split(':')[-1].strip()
                if gateway and '.' in gateway and ip and mask:
                    interface = f"{ip}/{mask}"
                    network = ipaddress.ip_interface(interface).network
                    return {'cidr': str(network), 'gateway': gateway}
        raise ValueError("No active network adapter with gateway found")
    except Exception as e:
        raise ValueError(f"Failed to fetch network info on Windows: {e}")
```

In [ ]:


def perform_arp_scan(ip_range):
    """
    Perform an ARP scan on the given IP range to discover connected devices.
    
    Crafts and sends ARP request packets, collects responses, and returns a list
    of devices with their IP and MAC addresses.
    
    Args:
        ip_range (str): The network range in CIDR notation (e.g., '192.168.1.0/24').
    
    Returns:
        list: A list of dictionaries, each with 'ip' and 'mac' keys.
    """
    arp = ARP(pdst=ip_range)
    ether = Ether(dst="ff:ff:ff:ff:ff:ff")
    packet = ether / arp
    result = srp(packet, timeout=3, verbose=0)[0]
    
    devices = []
    for sent, received in result:
        devices.append({'ip': received.psrc, 'mac': received.hwsrc})
    
    return devices

def print_devices(devices, gateway):
    """
    Print the list of discovered devices in a formatted table, highlighting the router.
    
    Args:
        devices (list): List of device dictionaries with 'ip' and 'mac'.
        gateway (str): The IP address of the router/default gateway.
    """
    print("Discovered Devices:")
    print(f"{'IP':<15} {'MAC':<17} {'Type':<10}")
    print("-" * 45)
    for device in devices:
        device_type = "(Router)" if device['ip'] == gateway else ""
        print(f"{device['ip']:<15} {device['mac']:<17} {device_type:<10}")
    print("-" * 45)

if __name__ == "__main__":
    os_type = detect_os()
    if os_type == 'unsupported':
        print("Unsupported OS. This script supports Linux and Windows only.")
    else:
        try:
            if os_type == 'linux':
                info = fetch_network_info_linux()
            elif os_type == 'windows':
                info = fetch_network_info_windows()
            
            ip_range = info['cidr']
            gateway = info['gateway']
            
            print(f"Scanning network: {ip_range}")
            print(f"Default Gateway (Router): {gateway}")
            
            devices = perform_arp_scan(ip_range)
            print_devices(devices, gateway)
        except ValueError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}. Ensure script is run with administrator/root privileges.")

Scanning network: 10.103.128.0/19
Default Gateway (Router): 10.103.128.1
Unexpected error: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may use conf.L3socket or conf.L3socket6 to access layer 3. Ensure script is run with administrator/root privileges.
